In [1]:
import numpy as np
import pandas as pd

In [2]:
path = [0, 1, 2, 3, 8, 10, 9, 7, 6, 5, 4, 0]
order = [0, 1, 2, 3, 10, 9, 8, 7, 4, 6, 5]
cumTime = [0, 14, 19, 22, 29, 39, 43, 47, 51, 56, 60, 73]
cumDistance = [0, 5.6, 7.4, 8.6, 10.7, 13.2, 14.2, 15.2, 16.1, 17.4, 18.1, 22.5]

In [3]:
def time_estimation(ggmap_time, numStop, numCustomer, timePerStop=5, timePerCustomer=2):

    estimation = ggmap_time * 1.1 + timePerStop * numStop + timePerCustomer * numCustomer
    estimation = estimation + (np.abs(np.random.normal(0, 0.1)) - 0.05) * estimation

    return np.round(estimation, 0)

In [4]:
df = pd.read_csv("./user-data.csv")
df = df.drop(['Unnamed: 0'], axis=1)

df.head()

,userID,orderID,addressID,parcelNumber
0,6500000001,1007,6,2
1,6500000002,1014,5,1
2,6500000003,1021,1,1
3,6500000004,1028,3,1
4,6500000005,1035,2,1


In [5]:
cumCustomer = [0] + list(df.groupby("addressID").count().userID.cumsum())

In [6]:
df["timeEstimation"] = df.apply(lambda x: time_estimation(cumTime[order[x.addressID]], order[x.addressID], cumCustomer[x.addressID]), axis = 1)

In [7]:
df.head()

,userID,orderID,addressID,parcelNumber,timeEstimation
0,6500000001,1007,6,2,221.0
1,6500000002,1014,5,1,214.0
2,6500000003,1021,1,1,55.0
3,6500000004,1028,3,1,111.0
4,6500000005,1035,2,1,69.0
